# Image Processing SS 20 - Assignment - 02

### Deadline is 6.5.2020 at 11:55am

Please solve the assignments together with a partner.
I will run every notebook. Make sure the code runs through. Select `Kernel` -> `Restart & Run All` to test it.


# Exercise 1 - 10 Points

Implement affine transformation with [bicubic interpolation](https://en.wikipedia.org/wiki/Bicubic_interpolation).
Implement the functions `affine_transformation` and `bicubic_interpolation`. Apply some affine transformation of your choice and smooth the output using your bicubic interpolation.

In [ ]:
# display the plots inside the notebook
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (12, 12)   # This makes the plot bigger

The [skimage](http://scikit-image.org/) library comes with multiple useful test images.  Let's start with an image of an astronaut. 

In [ ]:
from skimage.data import astronaut, coffee
from skimage.color import rgb2gray

In [ ]:
# We use a gray image. All the algorithms should work with color images too.
img = rgb2gray(coffee() / 255.)
plt.imshow(img, cmap='gray')
plt.show()

In [ ]:
def derive_y(image):
    """Computes the derivative of the image w.r.t the y coordinate"""
    derived_image = np.zeros_like(image)
    for x in range(image.shape[1]):
        for y in range(image.shape[0]):
            if y + 1 < image.shape[0] and y - 1 > 0:
                derived_image[y, x] = (image[y + 1, x] - image[y - 1, x]) / 2.0
    return derived_image

def derive_x(image):
    """Computes the derivative of the image w.r.t the x coordinate"""
    derived_image = np.zeros_like(image)
    for x in range(image.shape[1]):
        for y in range(image.shape[0]):
            if x + 1 < image.shape[1] and x - 1 > 0:
                derived_image[y, x] = (image[y, x + 1] - image[y, x - 1]) / 2.0
    return derived_image

In [ ]:
dx_img = derive_x(img)
dy_img = derive_y(img)

In [ ]:
plt.figure(figsize=(18, 12))
plt.subplot(131)
plt.imshow(img, cmap='gray')
plt.subplot(132)
plt.imshow(dx_img, cmap='gray')
plt.subplot(133)
plt.imshow(dy_img, cmap='gray')
plt.show()

Here are some sample affine transformations to be used later on

In [ ]:
T_scale = np.array([
    [1.75, 0, 0],
    [0, 1.75, 0],
    [0, 0, 1],
])

In [ ]:
T_affine = np.array([
    [1, 0.3, 0],
    [-0.3, 1, 0],
    [0, 0, 1],
])

In [ ]:
# you can use this function to invert the matrices
np.linalg.inv(T_scale)

In [ ]:
def affine_transformation(img, matrix, interpolation = True):
    XY=np.indices((img.shape[0], img.shape[1])).reshape(2, -1)
    # append homogenous coordinate
    XY = np.concatenate([XY, np.ones((1, XY.shape[1]))], axis=0)
    inv_t = np.linalg.inv(matrix)
    coords = inv_t @ XY
    # not needed for affine transformations, but for perspective transformations
    coords = np.true_divide(coords[:2, :], coords[[-1], :])
    if interpolation:
        return bicubic_interpolation(img, coords)

    coords = coords.astype(int)
    img_new = np.zeros_like(img).flatten()
    i = 0
    for x in range(img_new.shape[-1]):
        if 0 <= coords[0, i] < img.shape[0] and 0 <= coords[1, i] < img.shape[1]:
            img_new[i] = img[coords[0, i], coords[1, i]]
        i+=1
    return img_new.reshape(img.shape)
    

In [ ]:
def bicubic_interpolation(img, coords):
    dx_img = derive_x(img)
    dy_img = derive_y(img)
    dxy_img = derive_x(dy_img)
    result = np.zeros_like(img)
    result = result.flatten()
    X = np.zeros([16, result.shape[-1]])

    j = 0
    floor_coords = np.floor(coords).astype(int)
    ceil_coords = np.ceil(coords).astype(int)

    for i in range(coords.shape[-1]):
        x_val_floor = floor_coords[1, i]
        x_val_ceil = ceil_coords[1, i]
        y_val_floor = floor_coords[0, i]
        y_val_ceil = ceil_coords[0, i]

        if 0 <= x_val_floor < img.shape[1] and 0 <= x_val_ceil < img.shape[1] and 0 <= y_val_floor < img.shape[0] and 0 <= y_val_ceil < img.shape[0]:
            X[:, i] = np.array([
                img[y_val_floor][x_val_floor], img[y_val_ceil][x_val_floor], img[y_val_floor][x_val_ceil], img[y_val_ceil][x_val_ceil],
                dx_img[y_val_floor][x_val_floor], dx_img[y_val_ceil][x_val_floor], dx_img[y_val_floor][x_val_ceil], dx_img[y_val_ceil][x_val_ceil],
                dy_img[y_val_floor][x_val_floor], dy_img[y_val_ceil][x_val_floor], dy_img[y_val_floor][x_val_ceil], dy_img[y_val_ceil][x_val_ceil],
                dxy_img[y_val_floor][x_val_floor], dxy_img[y_val_ceil][x_val_floor], dxy_img[y_val_floor][x_val_ceil], dxy_img[y_val_ceil][x_val_ceil]
            ])
        j+=1

    inv_matrix = np.array([
            [ 1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
            [ 0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
            [-3,  3,  0,  0, -2, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
            [ 2, -2,  0,  0,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
            [ 0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0],
            [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0],
            [ 0,  0,  0,  0,  0,  0,  0,  0, -3,  3,  0,  0, -2, -1,  0,  0],
            [ 0,  0,  0,  0,  0,  0,  0,  0,  2, -2,  0,  0,  1,  1,  0,  0],
            [-3,  0,  3,  0,  0,  0,  0,  0, -2,  0, -1,  0,  0,  0,  0,  0],
            [ 0,  0,  0,  0, -3,  0,  3,  0,  0,  0,  0,  0, -2,  0, -1,  0],
            [ 9, -9, -9,  9,  6,  3, -6, -3,  6, -6,  3, -3,  4,  2,  2,  1],
            [-6,  6,  6, -6, -3, -3,  3,  3, -4,  4, -2,  2, -2, -2, -1, -1],
            [ 2,  0, -2,  0,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0],
            [ 0,  0,  0,  0,  2,  0, -2,  0,  0,  0,  0,  0,  1,  0,  1,  0],
            [-6,  6,  6, -6, -4, -2,  4,  2, -3,  3, -3,  3, -2, -1, -2, -1],
            [ 4, -4, -4,  4,  2,  2, -2, -2,  2, -2,  2, -2,  1,  1,  1,  1]
    ])

    alpha = inv_matrix @ X
    # 2 x N
    diffs = coords - floor_coords

    x = np.array([np.ones(diffs.shape[1]),
                      diffs[1, :],
                      np.power(diffs[1, :], 2.0),
                      np.power(diffs[1, :], 3.0)])
    y = np.array([np.ones(diffs.shape[1]),
                      diffs[0, :],
                      np.power(diffs[0, :], 2.0),
                      np.power(diffs[0, :], 3.0)])
    for i in range(result.shape[-1]):
        result[i] = x[:, i] @ alpha[:, i].reshape(4, 4) @ y[:, i].T
    result = result.reshape(img.shape)
    return result

Now show the results with and without smoothing.

In [ ]:
img_scale = affine_transformation(img, T_scale)
img_scale_no_interpolation = affine_transformation(img, T_scale, False)
img_affine = affine_transformation(img, T_affine)
img_affine_no_interpolation = affine_transformation(img, T_affine, False)

In [ ]:
plt.imshow(img_scale_no_interpolation, cmap='gray')
plt.show()
plt.imshow(img_scale, cmap='gray')
plt.show()

In [ ]:
plt.imshow(img_affine_no_interpolation, cmap='gray')
plt.show()
plt.imshow(img_affine, cmap='gray')
plt.show()